# pull_telemetry_example.ipynb
This notebook provides a tutorial for use of the Python `microSWIFTtelemetry` package.

Jake Davis (davisjr@uw.edu)\
Last updated: Dec 4, 2022

## Contents

`microSWIFTtelemetry` contains a collection of python functions for accessing data from the UW-APL SWIFT server:

http://swiftserver.apl.washington.edu/ (base URL)\
http://faculty.washington.edu/jmt3rd/SWIFTdata/DynamicDataLinks.html (HTML page)

The primary functions are contained in `pull_telemetry.py`. These include:

`pull_telemetry_as_var()` -- return request as a variable in memory\
`pull_telemetry_as_zip()` -- return request as a .zip file\
`pull_telemetry_as_json()` -- return request as JSON-formatted text\
`pull_telemetry_as_kml()` -- return a .kml file of GPS positions

Please see the doc strings in `pull_telemetry.py` for a full description of these functions and their arguments.

This file also contains the helper function `create_request()` which is not intended for use outside of the primary functions above.  The modules `read_sbd.py` and `compile_sbd.py` contain additional helper functions for unpacking data (in the form of Short Burst Data files) from the server.

## Installation:
The latest release of `microSWIFTtelemetry` can be installed from PyPI: 

``` bash
$ pip install microSWIFTtelemetry
```

## Examples:

### Specifying microSWIFT ID and date range:

All of the primary telemetry functions use the microSWIFT ID and a date range as base arguments. 

The ID should be defined as a string corresponding to the 3-digit microSWIFT name (including leading zeros). e.g. microSWIFT019:

In [ ]:
buoy_id = '019'

To query a list of IDs, current best practice us to create a list of IDs and loop over it. This makes it easier to handle the output, particularly when it is returned as a variable in memory.

In [ ]:
buoy_ids = ['019', '034', '057', '061']

The date range over which to query the server is defined using datetime objects:

In [ ]:
from datetime import datetime # import the datetime class from the datetime module

start = datetime(2022,9,26,0,0,0)
end = datetime(2022,9,29,0,0,0)

If the `end` date is left empty for any of the pull telemetry functions, it will default to `datetime.utcnow()`.

### Pull telemetry as a variable for a single buoy

The following examples demonstrate how to query the SWIFT server for microSWIFT data over a date range and 
return it as Python variable using `pull_telemetry_as_var()`. NOTE: the `.zip` file of short burst data (SBD) messages returned by the server is *handled in memory* and therefore is not saved to the local machine as a file. Use `pull_telemetry_as_zip()` for this purpose. 

If no data exists over the specified date range, an empty variable will be returned.

As a Python `dict`:

In [ ]:
from microSWIFTtelemetry import pull_telemetry_as_var

SWIFT_dict = pull_telemetry_as_var(buoy_id=buoy_id,
                                   start_date=start,
                                   end_date=end,
                                   var_type='dict')

print(SWIFT_dict.keys()) # e.g. SWIFT_dict['datetime']
print(len(SWIFT_dict['datetime'])) # print number of entries

Leaving the `end_date` argument empty will return any available data from `start_date` to the present time in UTC (though data may not be available up to the present time)

In [ ]:
from microSWIFTtelemetry import pull_telemetry_as_var

SWIFT_dict = pull_telemetry_as_var(buoy_id=buoy_id,
                                   start_date=start,
                                   var_type='dict')

print(SWIFT_dict.keys()) # e.g. SWIFT_dict['datetime']
print(len(SWIFT_dict['datetime'])) # print number of entries

As a `pandas.DataFrame`:

In [ ]:
from microSWIFTtelemetry import pull_telemetry_as_var

SWIFT_df = pull_telemetry_as_var(buoy_id=buoy_id,
                                 start_date=start,
                                 end_date=end,
                                 var_type='pandas')

print(SWIFT_df.info())

The microSWIFT DataFrame will have a `pandas.DatetimeIndex` that can be sliced using datetimes.

In [ ]:
from datetime import datetime, timezone

t1 = datetime(2022,9,28, tzinfo=timezone.utc)
t2 = datetime(2022,9,29, tzinfo=timezone.utc)

print(SWIFT_df[t1:t2].info())

In the future, other variable types (e.g. `xarray.Dataset`) will be supported.

### Pull telemetry as a variable for multiple buoys

For multiple microSWIFTs, run `pull_telemetry_as_var()` in a loop and store the individual variables in an iterable. For example, query the server for the microSWIFTs in `buoy_ids` (defined above) and store the output as individual Pandas DataFrames in a dictionary:

In [ ]:
from microSWIFTtelemetry import pull_telemetry_as_var

microSWIFTs = dict() # initialize a dict to store individual DataFrames

for buoy_id in buoy_ids:
    microSWIFTs[buoy_id] = pull_telemetry_as_var(buoy_id=buoy_id,
                                                 start_date=start,
                                                 end_date=end,
                                                 var_type='pandas')

print(microSWIFTs.keys())

The key-pair values will contain a DataFrame for each microSWIFT:

In [ ]:
microSWIFTs['019'].head()

### Pull telemetry as a zip file:

Query the SWIFT server for microSWIFT data over a specified date range and download a `.zip` file of individual short burst data (SBD) messages.

In [ ]:
from microSWIFTtelemetry import pull_telemetry_as_zip

pull_telemetry_as_zip(buoy_id=buoy_id,
                      start_date=start,
                      end_date=end)

The `.zip` file will be saved in your current directory with the microSWIFTs name, e.g. `microSWIFT019.zip`. If data exists, it the unzipped folder will contain the `.sbd` files.

For multiple microSWIFTs:

In [ ]:
from microSWIFTtelemetry import pull_telemetry_as_zip

for buoy_id in buoy_ids:
    pull_telemetry_as_zip(buoy_id=buoy_id,
                          start_date=start,
                          end_date=end)

### Pull telemetry as JSON text:

Query the SWIFT server and return a variable containing JSON-formatted text.

In [ ]:
from microSWIFTtelemetry import pull_telemetry_as_json

SWIFT_json = pull_telemetry_as_json(buoy_id='019',
                                    start_date=start,
                                    end_date=end)

print(SWIFT_json)

This variable can then be handled using the `json` package:

In [ ]:
import json

with open('SWIFT.json', 'w') as f:
    json.dump(SWIFT_json, f)

### Pull telemetry as a KML file

Query the SWIFT server for microSWIFT data over a specified date range and download a `.kml` file containing the buoy's GPS coordinates.  

In [ ]:
from microSWIFTtelemetry import pull_telemetry_as_kml

pull_telemetry_as_kml(buoy_id=buoy_id,
                      start_date=start,
                      end_date=end)

The `.kml` file will be saved in your current directory with the microSWIFTs name and the corresponding date range in ISO 8601 format (in UTC), e.g. `microSWIFT019_{y-m-dTH:M:S}_to_{y-m-dTH:M:S}-.kml`.

For multiple microSWIFTs:

In [ ]:
from microSWIFTtelemetry import pull_telemetry_as_kml

for buoy_id in buoy_ids:
    pull_telemetry_as_kml(buoy_id=buoy_id,
                          start_date=start,
                          end_date=end)

### Complete examples:

Pull telemetry from microSWIFT057 during Hurricane Ian and store it in memory as a Python dictionary. Extract the time and significant wave height as variables. Since the microSWIFTs were configured to report two wave estimates during this deployment, IMU-based (volt == 0) and GPS-based (volt == 1), filter for the GPS-based estimates and plot the result.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from microSWIFTtelemetry import pull_telemetry_as_var
from datetime import datetime 

buoy_id = '057'
start = datetime(2022,9,26,0,0,0)
end = datetime(2022,10,1,0,0,0)

SWIFT_dict = pull_telemetry_as_var(buoy_id=buoy_id,
                                   start_date=start,
                                   end_date=end,
                                   var_type='dict')

t = np.array(SWIFT_dict['datetime'])
Hs = np.array(SWIFT_dict['Hs'])
volt_flag = np.array(SWIFT_dict['volt'])


fig,ax = plt.subplots(1,1)
ax.scatter(t[volt_flag == 1], Hs[volt_flag == 1], label = 'GPS estimate')
ax.set_ylabel('Significant Wave Height (m)')
plt.xticks(rotation=30)
ax.legend()

Do the same as above, but using a more versatile Pandas DataFrame. 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from microSWIFTtelemetry import pull_telemetry_as_var
from datetime import datetime 

buoy_id = '057'
start = datetime(2022,9,26,0,0,0)
end = datetime(2022,10,1,0,0,0)

SWIFT_df = pull_telemetry_as_var(buoy_id=buoy_id,
                                 start_date=start,
                                 end_date=end,
                                 var_type='pandas')

SWIFT_GPS = SWIFT_df[SWIFT_df['volt'] == 1]

fig,ax = plt.subplots(1,1)
ax.scatter(SWIFT_GPS.index, SWIFT_GPS['Hs'], label = 'GPS estimate')
ax.set_ylabel('Significant Wave Height (m)')
plt.xticks(rotation=30)
ax.legend()